In [ ]:
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 42.9 MB/s eta 0:00:00


# Upload Any Fasta Refrence Genome you want to Align

In [ ]:
!apt-get -y install ncbi-blast+
!makeblastdb -in sequence.fasta -dbtype nucl -out local_hbv_reference_db


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ncbi-blast+ is already the newest version (2.12.0+ds-3build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Building a new DB, current time: 11/07/2024 20:44:19
New DB name:   /content/local_hbv_reference_db
New DB title:  sequence.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /content/local_hbv_reference_db
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1 sequences in 0.000357151 seconds.




# Helping Function

In [ ]:
def extract_regions(region_dicts, total_length):
  regions = []
  for region_dict in region_dicts:
      start = region_dict['query_start']
      end = region_dict['query_end']
      if 0 <= start < total_length and 0 < end <= total_length:
          regions.append((start, end))
  return regions
def find_uncovered_regions(region_dicts, total_length):
    """
    Finds regions that are not covered by the given region dictionaries.

    Args:
        region_dicts: A list of dictionaries, where each dictionary represents a region
                      with 'query_start' and 'query_end' keys.
        total_length: The total length of the sequence.

    Returns:
        A list of tuples, where each tuple represents an uncovered region (start, end).
    """
    covered_regions = extract_regions(region_dicts, total_length)  # Get the covered regions
    covered_regions.sort()  # Sort the regions by start position

    uncovered_regions = []
    last_end = 0  # Initialize the end of the last covered region to 0

    for start, end in covered_regions:
        if start > last_end:  # If there is a gap between the last covered region and the current one
            uncovered_regions.append((last_end, start))  # Add the gap as an uncovered region
        last_end = max(last_end, end)  # Update the end of the last covered region

    if last_end < total_length:  # If there is an uncovered region at the end
        uncovered_regions.append((last_end, total_length))  # Add it to the list

    return uncovered_regions
def hamming_distance(seq1, seq2):
    """Calculates the Hamming distance between two sequences."""
    if len(seq1) != len(seq2):
        raise ValueError("Sequences must have the same length")
    distance = sum(c1 != c2 for c1, c2 in zip(seq1, seq2))
    return distance

In [ ]:
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML


def complete_genome(hbv_genome_sequence,ref):

    #print(f"Length of query sequence: {len(hbv_genome_sequence)}")

    # Step 1: Save your sequence to a temporary file in FASTA format
    with open("hbv_genome_query.fasta", "w") as f:
        f.write(">HBV_genome\n")
        f.write(hbv_genome_sequence)

    # Step 2: Path to your local BLAST database (the reference HBV genome database)
    db_path = "local_hbv_reference_db"  # This is the BLAST database created from the reference genome

    # Step 3: Perform BLAST search for exact match (100% identity)
    blastn_cline = NcbiblastnCommandline(query="hbv_genome_query.fasta", db=db_path, evalue=0.001, outfmt=5, out="blast_output.xml",perc_identity=50)

    # Run the BLAST search and capture stdout/stderr
    stdout, stderr = blastn_cline()
    total_aligned=[]
    # Step 4: Parse the BLAST results to extract gene regions based on the alignment
    reference_genome = ""  # This will hold your entire reference genome sequence

    # Load the reference genome from your BLAST database (make sure you have access to it)
    with open(ref+".fasta", "r") as ref_file:
        reference_genome = "".join(ref_file.readlines()[1:]).strip()  # Get the sequence part (skip header)
    reference_genome=reference_genome.replace("\n","")

    # Parse the BLAST results and modify the reference genome
    with open("blast_output.xml") as result_handle:
        blast_records = NCBIXML.parse(result_handle)
        for record in blast_records:

            for alignment in record.alignments:
                for hsp in alignment.hsps:
                    # Replace the aligned portion of the reference genome with the hbv_genome_sequence
                    start_pos = hsp.sbjct_start - 1  # Convert to 0-based indexing
                    end_pos = hsp.sbjct_end  # End position is inclusive, so no change needed
                    query_start = hsp.query_start -1
                    query_end = hsp.query_end
                   # print(query_start,query_end,start_pos,end_pos)
                    # Construct the modified reference genome with the aligned region replaced by the query sequence
                    modified_genome = (
                        reference_genome[:start_pos] +  # Part before the alignment
                        hbv_genome_sequence[query_start:query_end] +  # Replace the aligned region with the query sequence
                        reference_genome[end_pos:]  # Part after the alignment
                    )
                    total_aligned.append({'query_start':query_start,'query_end':query_end})
    try:
      return modified_genome,total_aligned,reference_genome
    except:
      return modified_genome,total_aligned,reference_genome



/usr/local/lib/python3.10/dist-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


In [ ]:
def modified_data(input,reference_genome='sequence'):

    modified_genome,total_aligned,reference_genome=complete_genome(input,reference_genome)
    try:
        uncovered_regions=find_uncovered_regions(total_aligned,len(input))
        seq1 = modified_genome #[0:3182]
        seq2 = reference_genome #[0:3182]
        distance = hamming_distance(seq1, seq2)
        dissimilarity_percentage = (distance / len(seq1)) * 100
        return modified_genome,dissimilarity_percentage,uncovered_regions
    except:
        return modified_genome,0.00,uncovered_regions

# Download HBV Sequence from NCBI Repository using API

In [ ]:
import pandas as pd
import pandas as pd
from Bio import Entrez
import csv

def download_sequences_with_metadata(query, num_sequences, output_file):
    # Set up Entrez email address (required by NCBI)
    Entrez.email = 'your_email@example.com'

    # Perform the search on NCBI's Nucleotide database
    handle = Entrez.esearch(db='nucleotide', term=query, retmax=num_sequences)
    search_results = Entrez.read(handle)
    handle.close()

    # Fetch the sequences along with metadata based on the search results
    id_list = search_results['IdList']
    handle = Entrez.efetch(db='nucleotide', id=id_list, rettype='gb', retmode='xml')
    records = Entrez.read(handle)
    handle.close()

    # Open a CSV file to write the sequences and metadata
    with open(output_file, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write header with metadata columns
        writer.writerow(['Sequence ID', 'Sequence', 'Isolation Source', 'Genotype', 'Collection Date', 'Organism', 'Accession Number','Gene','loc'])

        # Iterate over the records to extract metadata and sequence
        for record in records:
            sequence_id = record.get('GBSeq_locus', 'N/A')
            sequence = record.get('GBSeq_sequence', 'N/A')
            isolation_source = extract_qualifier(record, 'isolation_source')
            genotype = extract_qualifier(record, 'genotype')
            collection_date = extract_qualifier(record, 'collection_date')
            organism = record.get('GBSeq_organism', 'N/A')
            accession_number = record.get('GBSeq_accession-version', 'N/A')
            gene_name = extract_qualifier(record, 'gene')
            location = extract_qualifier(record, 'note')

            # Write the data to the CSV file
            writer.writerow([sequence_id, sequence, isolation_source, genotype, collection_date, organism, accession_number,gene_name,location])

    print(f'{len(records)} sequences downloaded and saved to {output_file}.')

def extract_qualifier(record, qualifier_name):
    """
    Extracts a specific qualifier from a GenBank record.
    """
    for feature in record.get('GBSeq_feature-table', []):
        for qualifier in feature.get('GBFeature_quals', []):
            if qualifier.get('GBQualifier_name') == qualifier_name:
                return qualifier.get('GBQualifier_value', 'N/A')
    return 'N/A'

# Example usage
query = 'HBV AND HCC'
num_sequences = 100000  # Adjust as needed
output_file = 'hcc1.csv'

download_sequences_with_metadata(query, num_sequences, output_file)

In [ ]:


query = 'hbv carcinoma tumor'
num_sequences = 100000  # Adjust as needed
output_file = 'hcc2.csv'

download_sequences_with_metadata(query, num_sequences, output_file)


1010 sequences downloaded and saved to hcc2.csv.


# Download Data from Repository

In [ ]:
import gdown
import pandas as pd

# Define the Google Drive public link
url = 'https://drive.google.com/uc?id=1jFJ1Xe8ZBwxQWIhrn3hHjGhZZA1s3izT'

# Define the file name to save the DataFrame
output_file = 'final.csv'

# Download the file from the Google Drive link
gdown.download(url, output_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1jFJ1Xe8ZBwxQWIhrn3hHjGhZZA1s3izT
To: /content/final.csv
100%|██████████| 75.3M/75.3M [00:01<00:00, 73.9MB/s]


'final.csv'

In [ ]:
import pandas as pd



dataset=pd.read_csv("final.csv")
dataset= dataset.loc[dataset['Vrius_Gene'].isin(['X', 'S','C','P'])]#.sample(1000)



dataset=dataset.dropna(subset=['Disease'])
dataset=dataset.drop_duplicates(subset=['Sequence'])

In [ ]:
hcc1 = pd.read_csv('hcc1.csv')

# Load the second file into a DataFrame.
hcc2 = pd.read_csv('hcc2.csv')

# Merge the two DataFrames.
concatenated_df = pd.concat([hcc1, hcc2], axis=0, ignore_index=True)
organisms_to_exclude = ['Homo sapiens', 'Mus musculus', 'Rattus norvegicus','Hepacivirus hominis',
                        'Larimichthys crocea', 'Triplophysa tibetana',
                        'Collichthys lucidus', 'Macaca mulatta', 'Bos taurus']

# Drop rows containing the specified organisms
filtered_df = concatenated_df[~concatenated_df['Organism'].isin(organisms_to_exclude)].dropna(subset='Isolation Source')

# View the filtered data
filtered_df.head()
organisms_to_exclude = ['non-tumor tissue', 'HBsAg positive non-HCC control','serum of HBsAg-positive patient', 'serum',
       'serum of HBsAg-negative patient',
       'serum of HBsAg-negative subject (control)', 'biopsy',
       'cirrhotic patient','HBsAg negative non-HCC control','Shiraz hospitals']

# Drop rows containing the specified organisms
filtered_df = filtered_df[~filtered_df['Isolation Source'].isin(organisms_to_exclude)]
filtered_df=filtered_df.drop_duplicates(subset='Sequence ID') #['Organism'].unique()
filtered_df['length']=filtered_df['Sequence'].str.len()
filtered_df['Sequence'] = filtered_df['Sequence'].str.upper()
filtered_df['labels']=1
#filtered_df=filtered_df[['Sequence','labels']]
filtered_df=filtered_df.rename(columns={'Sequence':'seq'})
filtered_df['Disease']='HCC'
iso_to_exclude = ['non-tumor tissue from patient 11','non-tumor tissue from patient 9','non-tumor tissue from patient 10','tumor tissue from patient 11']

# Drop rows containing the specified organisms
filtered_df = filtered_df[~filtered_df['Isolation Source'].isin(iso_to_exclude)]
iso_to_exclude = ['Non-HCC']

# Drop rows containing the specified organisms
filtered_df=filtered_df[~filtered_df['loc'].str.contains('Non-HCC', na=False)]
filtered_df=filtered_df[~filtered_df['Isolation Source'].str.contains('Isolation Source', na=False)]
filtered_data = filtered_df[
    filtered_df['Isolation Source'].str.contains('HCC|tumor', case=False, na=False) |
    filtered_df['loc'].str.contains('HCC|tumor', case=False, na=False)
]
filtered_data=filtered_data.drop_duplicates(subset='seq')

In [ ]:
dataset=dataset[['Sequence',
       'Sequence_description',   'Virus_genotype',
        'Isolation_source',  'Disease',
       'Vrius_Gene']]
filtered_df=filtered_df[['seq',
       'loc',   'Genotype',
        'Isolation Source',  'Disease',
       'Gene']]
filtered_df.columns=['Sequence',
       'Sequence_description',   'Virus_genotype',
        'Isolation_source',  'Disease',
       'Vrius_Gene']
import pandas as pd

final=pd.concat([dataset,filtered_df],axis=0,ignore_index=True)

In [ ]:
# final=final.loc[final['Disease'] == 'HCC'	]

In [ ]:
final=final.drop_duplicates(subset='Sequence')

In [ ]:
mgenomes = []
dissimilarities = []
uncovered_regions = []
index=[]
extracted_data=pd.DataFrame(columns=['Sequence',
       'Sequence_description',   'Virus_genotype',
        'Isolation_source',  'Disease',
       'Vrius_Gene'])
import tqdm
all_rows = []
for i in tqdm.tqdm(range(len(final['Sequence'].values))):
      try:
        mgenome,dissimilarity,uncovered_reg=modified_data(final['Sequence'].values[i])
        row_data = final.iloc[i, :].to_dict()
        row_data['seq'] = mgenome
        row_data['dissimilarity'] = dissimilarity
        row_data['uncovered_reg'] = uncovered_reg
        all_rows.append(row_data)
      except:
        print("Something Wrong")

extracted_data = pd.concat([extracted_data, pd.DataFrame(all_rows)], ignore_index=True)


In [ ]:
import pandas as pd
extracted_data=pd.read_csv("/content/drive/MyDrive/Upwork_Projects/HBV/Experiment1/Dataset/hbv_aligned_data(HCC_NHCC).csv")

In [ ]:
duplicates_mask=extracted_data[extracted_data['Disease'] == 'HCC'].duplicated(subset=['seq'])
test=extracted_data[extracted_data['Disease'] == 'HCC'][duplicates_mask] #['Sequence_description'].value_counts() #['Sequence'] #.apply(len).value_counts() #.drop_duplicates('seq')['Sequence_description'].value_counts()
test.to_csv("/content/drive/MyDrive/Upwork_Projects/HBV/Experiment1/Dataset/test_dataset.csv")

In [ ]:
extracted_data.to_csv("/content/drive/MyDrive/Upwork_Projects/HBV/Experiment1/Dataset/dataset.csv")

In [ ]:
# Create a boolean mask for rows where 'Disease' is 'HCC'
nhcc_mask = extracted_data['Disease'] != 'HCC'

# Apply drop_duplicates only to the subset of rows with 'HCC'
extracted_data2 =   pd.concat([extracted_data[~nhcc_mask], extracted_data[nhcc_mask].drop_duplicates(subset=['seq'], keep='first')], ignore_index=True).reset_index(drop=True)